In [66]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=servico)

In [68]:
import pandas as pd
import os
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.alert import Alert
from time import sleep

df = pd.read_excel('Processos.xlsx')

caminho = os.getcwd()
arquivo = 'file://' + caminho + '/index.html'

driver.get(arquivo)

for i, processo in enumerate(df['Nome']):
    menu = driver.find_element(By.XPATH, '/html/body/div/div/button')
    estados = driver.find_elements(By.CSS_SELECTOR, 'div > a')
    
    estados = {
        'Distrito Federal': estados[0],
        'Rio de Janeiro': estados[1],
        'São Paulo': estados[2]
    }

    ActionChains(driver).move_to_element(menu).perform()
    estados[df['Cidade'][i]].click()

    aba_original = driver.window_handles[0]
    nova_aba = driver.window_handles[1]
    driver.switch_to.window(nova_aba)

    nome = df['Nome'][i]
    driver.find_element(By.XPATH, '//*[@id="nome"]').send_keys(nome)

    advogado = df['Advogado'][i]
    driver.find_element(By.XPATH, '//*[@id="advogado"]').send_keys(advogado)

    processo = df['Processo'][i]
    driver.find_element(By.XPATH, '//*[@id="numero"]').send_keys(processo)

    driver.find_element(By.XPATH, '//*[@id="formulario"]/div/button').click()

    alerta = Alert(driver)
    alerta.accept()
    
    while True:
        try:
            alerta = driver.switch_to.alert
            break
        except:
            sleep(1)
    texto_alerta = alerta.text
    
    if "Processo encontrado com sucesso" in texto_alerta:
        df['Status'][i] = "Encontrado"
        alerta.accept()
    else:
        df['Status'][i] = "Não encontrado"
        alerta.accept()
    
    driver.close()

    driver.switch_to.window(aba_original)

/tmp/ipykernel_193167/1394788077.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Status'][i] = "Encontrado"


In [69]:
driver.quit()
display(df)

df.to_excel('Processos Atualizado.xlsx')

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Encontrado
3,Carol,Amanda,PC5197,São Paulo,Encontrado
